![](../storage/banners/13_tmp.png)

### <span style="color:#0ab49a">Проверка dynamics.py.</span> <span style="color:#A254FC">Фильтр Калмана: генерация файлика</span> 

In [1]:
from storage.observability_mapping_partial_derivatives import h_element, h_matrix
import numpy as np
import sympy as sp

In [2]:
# cf {i_c} {i_f} {jj} {ii} {send_len} {take_len}
a = []
with open(f"storage/measures_vector_notes_last.txt", "r") as f:
    for l in f:
        tmp = l.split()
        if tmp[0] != "#":
            a += [{"rel": tmp[0], 'i_12': [int(tmp[1]), int(tmp[2])], 'jj': int(tmp[3]), 'ii': int(tmp[4]), 'send': int(tmp[5]), 'take': int(tmp[6])}]
        elif tmp[1] == "Параметр":
            if tmp[2] == "CUBESAT_AMOUNT":
                cn = int(tmp[3])
                print(f"Количество кубсатов: {cn}")
            if tmp[2] == "CHIPSAT_AMOUNT":
                fn = int(tmp[3])
                print(f"Количество чипсатов: {fn}")
            if tmp[2] == "NAVIGATION_ANGLES":
                angles_navigation = bool(tmp[3])
                print(f"Угловая навигация: {angles_navigation}")
            if tmp[2] == "N_ANTENNA_C":
                c_ant = int(tmp[3])
                print(f"Антенн на кубсате: {c_ant}")
            if tmp[2] == "N_ANTENNA_F":
                f_ant = int(tmp[3])
                print(f"Антенн на чипсате: {f_ant}")
            if tmp[2] == "MULTI_ANTENNA_TAKE":
                multy_antenna_take = bool(tmp[3])
                print(f"Приём с нескольких антенн: {multy_antenna_take}")
            if tmp[2] == "MULTI_ANTENNA_SEND":
                multy_antenna_send = bool(tmp[3])
                print(f"Отправка с нескольких антенн: {multy_antenna_send}")

Количество кубсатов: 4
Количество чипсатов: 3
Угловая навигация: True
Антенн на кубсате: 1
Антенн на чипсате: 1
Приём с нескольких антенн: True
Отправка с нескольких антенн: True


In [3]:
r_f = [np.random.uniform(-100, 100, 3) for i in range(fn)]
r_c = [np.random.uniform(-10, 10, 3) for i in range(cn)]
q_f = [np.random.uniform(-10, 10, 4) for i in range(fn)]
q_c = [np.random.uniform(-10, 10, 4) for i in range(cn)]
q_f = [q_f[i]/np.linalg.norm(q_f[i]) for i in range(fn)]
q_c = [q_c[i]/np.linalg.norm(q_c[i]) for i in range(cn)]

w_0 = 1e-4
t = 0.

def beauty_sympy(exp, numlen = 1):
    return exp.applyfunc(lambda x: sp.Symbol(f"{x:.{numlen}f}") if abs(float(x))>1e-6 else 0)

H, top, bot = h_matrix(c_ant=c_ant, f_ant=f_ant, fn=fn, cn=cn, angles_navigation=angles_navigation, r_f=r_f, r_c=r_c, q_f=q_f, q_c=q_c, multy_antenna_send=multy_antenna_send, multy_antenna_take=multy_antenna_take, w_0=w_0, t=t)
beauty_sympy(sp.Matrix(H))

ValueError: too many values to unpack (expected 3)

In [4]:
# print(f"Верхняя подматрица:")
# beauty_sympy(sp.Matrix(top))

In [5]:
# if bot is not None:
#     print(f"Нижняя подматрица:")
#     beauty_sympy(sp.Matrix(bot))

##### **Проверка H: Шаблон**

In [6]:
H_template, _, _ = h_matrix(c_ant=c_ant, f_ant=f_ant, fn=fn, cn=cn, angles_navigation=angles_navigation, r_f=r_f, r_c=r_c, q_f=q_f, q_c=q_c, multy_antenna_send=multy_antenna_send, multy_antenna_take=multy_antenna_take, w_0=w_0, t=t, return_template=True)
sp.Matrix(H_template)

Matrix([
[cf_0^0, cf_0^0, cf_0^0],
[cf_0^1, cf_0^1, cf_0^1],
[cf_0^2, cf_0^2, cf_0^2],
[cf_1^0, cf_1^0, cf_1^0],
[cf_1^1, cf_1^1, cf_1^1],
[cf_1^2, cf_1^2, cf_1^2],
[cf_2^0, cf_2^0, cf_2^0],
[cf_2^1, cf_2^1, cf_2^1],
[cf_2^2, cf_2^2, cf_2^2],
[cf_3^0, cf_3^0, cf_3^0],
[cf_3^1, cf_3^1, cf_3^1],
[cf_3^2, cf_3^2, cf_3^2],
[ff_1^0, ff_1^0, ff_1^0],
[ff_2^0, ff_2^0, ff_2^0],
[ff_2^1, ff_2^1, ff_2^1]])

##### **Проверка H: сравнение строк измерений**

In [7]:
a

[{'rel': 'fc', 'i_12': [0, 0], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [0, 0], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'fc', 'i_12': [1, 0], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [0, 1], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'fc', 'i_12': [2, 0], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [0, 2], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'fc', 'i_12': [0, 1], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [1, 0], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'fc', 'i_12': [1, 1], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [1, 1], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'fc', 'i_12': [2, 1], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [1, 2], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'fc', 'i_12': [0, 2], 'jj': 0, 'ii': 0, 'send': 1, 'take': 1},
 {'rel': 'cf', 'i_12': [2, 0], 'jj': 0, 'ii': 0, 'send': 1, 'tak

In [8]:
"""c_ant = 1
f_ant = 1
cn = 1
fn = 5
angles_navigation = True"""

tmp = 1e4
H_mask = None
for i in a:
    i_fs = [i['i_12'][i['rel'].index('f')]] if 'c' in i['rel'] else i['i_12']
    line = []
    for j in range(fn):
        if angles_navigation:
            line.extend([tmp if j in i_fs else 0 for _ in range(7)] + [0]*6)
        else:
            line.extend([tmp if j in i_fs else 0 for _ in range(3)] + [0]*3)
    line = np.array([line])
    H_mask = line if H_mask is None else np.vstack([H_mask, line])

# beauty_sympy(sp.Matrix(H_mask))

In [9]:
beauty_sympy(sp.Matrix(H + H_mask))

Matrix([
[10000.8, 10000.9, 9999.6,  9966.7,  9982.1, 10074.0,  9888.5, 0, 0, 0, 0, 0, 0,       0,       0,       0,       0,       0,       0,       0, 0, 0, 0, 0, 0, 0,       0,       0,       0,      0,      0,       0,      0, 0, 0, 0, 0, 0, 0],
[10000.8, 10000.9, 9999.6,  9966.7,  9982.1, 10074.0,  9888.5, 0, 0, 0, 0, 0, 0,       0,       0,       0,       0,       0,       0,       0, 0, 0, 0, 0, 0, 0,       0,       0,       0,      0,      0,       0,      0, 0, 0, 0, 0, 0, 0],
[      0,       0,      0,       0,       0,       0,       0, 0, 0, 0, 0, 0, 0, 10001.1, 10000.3,  9999.4,  9963.7,  9977.9, 10040.8, 10031.9, 0, 0, 0, 0, 0, 0,       0,       0,       0,      0,      0,       0,      0, 0, 0, 0, 0, 0, 0],
[      0,       0,      0,       0,       0,       0,       0, 0, 0, 0, 0, 0, 0, 10001.1, 10000.3,  9999.4,  9963.7,  9977.9, 10040.8, 10031.9, 0, 0, 0, 0, 0, 0,       0,       0,       0,      0,      0,       0,      0, 0, 0, 0, 0, 0, 0],
[      0,       0,      0, 

In [42]:
s = """[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.00000000e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.00000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.26623828e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.28395898e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.85187694e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.26623828e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.00000000e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  5.00000000e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.26623828e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.28395898e-06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.85187694e-06  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.26623828e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]"""
s = s.replace("\n", "")
s = s.replace("  -", ", -")
s = s.replace(" -", ", -")
s = s.replace("   ", ", ")
s = s.replace("  ", ", ")
s = s.replace("]", "],")
s = s.replace(",,", ",")
s = s[:-1]
M = eval(s)
print("Матрица Ф:")

beauty_sympy(sp.Matrix(M), numlen=2)

Матрица Ф:


Matrix([
[0,     0,    0, 0, 0, 0, 1.00,    0,     0,    0,    0,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0,     0,    0, 0, 0, 0,    0, 1.00,     0,    0,    0,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0,     0,    0, 0, 0, 0,    0,    0,  1.00,    0,    0,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0,     0,    0, 0, 0, 0,    0,    0,     0, 0.50,    0,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0,     0,    0, 0, 0, 0,    0,    0,     0,    0, 0.50,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0, 0.50, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0,     0,    0, 0, 0, 0,    0,    0, -0.00,    0,    0,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0],
[0, -0.00,    0, 0, 0, 0,    0,    0,     0,    0,    0,    0, 0,     0,    0, 0, 0, 0,    0,    0,     0,   